# Adding an Item with the Omeka S API

The "basics" of accessing the API, including authentication and querying item lists, are covered in the previous notebook
[`omeka-s-api-basics.ipynb`](omeka-s-api-basics.ipynb).

For some (dated 2019) advice on doing this with `curl` see: https://forum.omeka.org/t/example-api-usage-using-curl/8083. 

Since this is a standalone notebook, go through the setup and authentication again:

In [1]:
import requests
import json

In [2]:
# establish API location/endpoints

siteUrl = 'http://jajohnst.si676.si.umich.edu/omeka-s' # if you replicate this example, provide the URI for your site
endpoint = '/api'

In [ ]:
def get_credentials(credential_file_path):
    '''Retrieve Omeka S Api credentials from another file. 
    That file must be a JSON file.'''

    with open(credential_file_path, 'r') as credentials:
        keys = json.load(credentials)
    
    return keys['key_identity'], keys['key_credential']

key_identity, key_credential = get_credentials('../collection-project/omeka-credentials.json')

In [ ]:
print(key_credential, key_identity)

## Use the Omeka S API to add a new item

The process involves the data for the new item, then
using a `post` request to add the item via the API. 

In [4]:
action = '/items'

parameters = {
    'key_credential': key_credential,
    'key_identity'  : key_identity
}

headers = {
    'Content-Type': 'application/json'
}

In [5]:
# create data for a new item

data = { 
    "dcterms:title": [{
        "property_id": 1,
        "property_label": "Title",
        "@value": "A Mere Title for an item created via the API",
        "type": "literal",
        "@language": "en-us"
        }],
    "dcterms:rights": [{
        "type": "literal",
        "property_id": 15,
        "property_label": "Rights",
        "is_public": True,
        "@value": "No known restrictions on publication.",
        "@language": "en-us"
        }]
    }

In [6]:
# now, post the data

post_item_url = siteUrl + endpoint + action

new_item_post = requests.post(post_item_url, headers=headers, params=parameters, data=json.dumps(data))

print(new_item_post.url,new_item_post.status_code,'\n',new_item_post.headers)

http://jajohnst.si676.si.umich.edu/omeka-s/api/items?key_credential=nj0K6JAqnFgeqyP3Kq40SOhl6fiPBXky&key_identity=yAu5NAEEtJCvGx4s8XgJSgvTFjwvLD6j 200 
 {'Date': 'Tue, 19 Nov 2024 16:37:51 GMT', 'Server': 'Apache', 'Omeka-S-Version': '4.1.1', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/ld+json'}


In [7]:
new_item_post.json()

{'@context': 'http://jajohnst.si676.si.umich.edu/omeka-s/api-context',
 '@id': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/items/454',
 '@type': 'o:Item',
 'o:id': 454,
 'o:is_public': True,
 'o:owner': {'@id': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/users/1',
  'o:id': 1},
 'o:resource_class': None,
 'o:resource_template': None,
 'o:thumbnail': None,
 'o:title': 'A Mere Title for an item created via the API',
 'thumbnail_display_urls': {'large': None, 'medium': None, 'square': None},
 'o:created': {'@value': '2024-11-19T16:37:52+00:00',
  '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
 'o:modified': {'@value': '2024-11-19T16:37:52+00:00',
  '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
 'o:primary_media': None,
 'o:media': [],
 'o:item_set': [],
 'o:site': [{'@id': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/sites/1',
   'o:id': 1},
  {'@id': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/sites/2',
   'o:id': 2}],
 'dcterms:title': [{'type': 'literal'

## Use the Omeka S API to add a new item with Media

The process requires the metadata for the new item,
the link to the file (aka "media" in Omeka S terms), and
then using a `post` request to add the item via the API. 

In [8]:
# example of how you can format the metadata
# note that this is a python dictionary, and can then be converted in valid JSON

post_data = {
    "dcterms:title": [
        {
            "property_id": 1,
            "property_label": "Title",
            "@value": "An image of an Orca, archived from an old website, and uploaded via the API",
            "type": "literal"
        }
    ],
    "@type": "o:Item",
    "o:item_set": [
        {
            "o:id": 440
        }
    ],
    "o:media": [
        {
            "o:ingester": "upload",
            "file_index": "0",
            "o:item": {},
            "dcterms:title": [
                {
                    "property_id": 1,
                    "property_label": "Title",
                    "@value": "Media file for the archived Orca image",
                    "type": "literal"
                }
            ]
        }
    ]
}

## Trying with `curl`

See these examples of using the Omeka S API with `curl`: https://forum.omeka.org/t/example-api-usage-using-curl/8083

In [9]:
post_data = { "dcterms:title" : [ {"property_id": 1, "property_label" : "Title", "@value" : "An image of an Orca, archived from an old website, and uploaded via the API", "type" : "literal" } ], "@type" : "o:Item", "o:item_set" : [ {"o:id": 440}], "o:media" : [{"o:ingester": "upload", "file_index": "0", "o:item": {}, "dcterms:title" : [ { "property_id" : 1, "property_label" : "Title", "@value" : "Media file for the archived Orca image", "type" : "literal" } ]}] }

print(json.dumps(post_data)) # this converts the data into valid JSON

{"dcterms:title": [{"property_id": 1, "property_label": "Title", "@value": "An image of an Orca, archived from an old website, and uploaded via the API", "type": "literal"}], "@type": "o:Item", "o:item_set": [{"o:id": 440}], "o:media": [{"o:ingester": "upload", "file_index": "0", "o:item": {}, "dcterms:title": [{"property_id": 1, "property_label": "Title", "@value": "Media file for the archived Orca image", "type": "literal"}]}]}


This example didn't work, probably because the JSON elements were not formatted correctly:

```bash
curl -v -F "data={'dcterms:title': [{'property_id': 1, 'property_label': 'Title', '@value': 'An image of an Orca, archived from an old website, and uploaded via the API', 'type': 'literal'}], '@type': 'o:Item', 'o:item_set': [{'o:id': 441}], 'o:media': [{'o:ingester': 'upload', 'file_index': '0', 'o:item': {}, 'dcterms:title': [{'property_id': 1, 'property_label': 'Title', '@value': 'Media file for the archived Orca image', 'type': 'literal'}]}]}" -F "file[0]=@./data/webfiles-samples/image/orca.via_.moc_.noaa_.jpg" "http://jajohnst.si676.si.umich.edu/omeka-s/api/items?key_credential=nj0K6JAqnFgeqyP3Kq40SOhl6fiPBXky&key_identity=yAu5NAEEtJCvGx4s8XgJSgvTFjwvLD6j"
```

Produced this response:

```
* processing: http://jajohnst.si676.si.umich.edu/omeka-s/api/items?key_credential=nj0K6JAqnFgeqyP3Kq40SOhl6fiPBXky&key_identity=yAu5NAEEtJCvGx4s8XgJSgvTFjwvLD6j
*   Trying 141.211.145.90:80...
* Connected to jajohnst.si676.si.umich.edu (141.211.145.90) port 80
> POST /omeka-s/api/items?key_credential=nj0K6JAqnFgeqyP3Kq40SOhl6fiPBXky&key_identity=yAu5NAEEtJCvGx4s8XgJSgvTFjwvLD6j HTTP/1.1
> Host: jajohnst.si676.si.umich.edu
> User-Agent: curl/8.2.1
> Accept: */*
> Content-Length: 83277
> Content-Type: multipart/form-data; boundary=------------------------16ca63cbdb399460
> 
* We are completely uploaded and fine
< HTTP/1.1 400 Bad Request
< Date: Tue, 19 Nov 2024 15:05:24 GMT
< Server: Apache
< Omeka-S-Version: 4.1.1
< Connection: close
< Transfer-Encoding: chunked
< Content-Type: application/ld+json
< 
* Closing connection
{"errors":{"error":"JSON: Syntax error"}}%
```

### Working `curl` request

```bash
curl -v -F 'data={"dcterms:title": [{"property_id": 1, "property_label": "Title", "@value": "An image of an Orca, archived from an old website, and uploaded via the API", "type": "literal"}], "@type": "o:Item", "o:item_set": [{"o:id": 441}], "o:media": [{"o:ingester": "upload", "file_index": "0", "o:item": {}, "dcterms:title": [{"property_id": 1, "property_label": "Title", "@value": "Media file for the archived Orca image", "type": "literal"}]}]}' -F "file[0]=@./data/webfiles-samples/image/orca.via_.moc_.noaa_.jpg" "http://jajohnst.si676.si.umich.edu/omeka-s/api/items?key_credential=nj0K6JAqnFgeqyP3Kq40SOhl6fiPBXky&key_identity=yAu5NAEEtJCvGx4s8XgJSgvTFjwvLD6j"
```

### Uploading the media item with Python requests

Using the working `curl` command, trying this curl converter to python requests... 

https://curlconverter.com/

In [ ]:
import requests

In [10]:
siteAPI = 'http://jajohnst.si676.si.umich.edu/omeka-s/api'
action = '/items'

params = {
    'key_credential': 'nj0K6JAqnFgeqyP3Kq40SOhl6fiPBXky',
    'key_identity': 'yAu5NAEEtJCvGx4s8XgJSgvTFjwvLD6j',
}

files = {
    'data': (None, '{"dcterms:title": [{"property_id": 1, "property_label": "Title", "@value": "An image of an Orca, archived from an old website, and uploaded via the API", "type": "literal"}], "@type": "o:Item", "o:item_set": [{"o:id": 440}], "o:media": [{"o:ingester": "upload", "file_index": "0", "o:item": {}, "dcterms:title": [{"property_id": 1, "property_label": "Title", "@value": "Media file for the archived Orca image", "type": "literal"}]}]}'),
    'file[0]': open('../data/webfiles-samples/image/orca.via_.moc_.noaa_.jpg', 'rb'),
}

r = requests.post(siteAPI + action, params=params, files=files)

print(r.url, r.status_code)

http://jajohnst.si676.si.umich.edu/omeka-s/api/items?key_credential=nj0K6JAqnFgeqyP3Kq40SOhl6fiPBXky&key_identity=yAu5NAEEtJCvGx4s8XgJSgvTFjwvLD6j 200


In [11]:
r.json()

{'@context': 'http://jajohnst.si676.si.umich.edu/omeka-s/api-context',
 '@id': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/items/455',
 '@type': 'o:Item',
 'o:id': 455,
 'o:is_public': True,
 'o:owner': {'@id': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/users/1',
  'o:id': 1},
 'o:resource_class': None,
 'o:resource_template': None,
 'o:thumbnail': None,
 'o:title': 'An image of an Orca, archived from an old website, and uploaded via the API',
 'thumbnail_display_urls': {'large': 'http://jajohnst.si676.si.umich.edu/omeka-s/files/large/de626197ce76c28ac4d55958025a8d662b8cd9a8.jpg',
  'medium': 'http://jajohnst.si676.si.umich.edu/omeka-s/files/medium/de626197ce76c28ac4d55958025a8d662b8cd9a8.jpg',
  'square': 'http://jajohnst.si676.si.umich.edu/omeka-s/files/square/de626197ce76c28ac4d55958025a8d662b8cd9a8.jpg'},
 'o:created': {'@value': '2024-11-19T16:40:00+00:00',
  '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
 'o:modified': {'@value': '2024-11-19T16:40:00+00:00',
  '@

Try uploading without property ID:

In [12]:
siteAPI = 'http://jajohnst.si676.si.umich.edu/omeka-s/api'
action = '/items'

params = {
    'key_credential': 'nj0K6JAqnFgeqyP3Kq40SOhl6fiPBXky',
    'key_identity': 'yAu5NAEEtJCvGx4s8XgJSgvTFjwvLD6j',
}

files = {
    'data': (None, '{"dcterms:title": [{"property_label": "Title", "@value": "An image of an Orca, archived from an old website, and uploaded via the API", "type": "literal"}], "@type": "o:Item", "o:item_set": [{"o:id": 440}], "o:media": [{"o:ingester": "upload", "file_index": "0", "o:item": {}, "dcterms:title": [{"property_label": "Title", "@value": "Media file for the archived Orca image", "type": "literal"}]}]}'),
    'file[0]': open('../data/webfiles-samples/image/orca.via_.moc_.noaa_.jpg', 'rb'),
}

r = requests.post(siteAPI + action, params=params, files=files)

print(r.url, r.status_code)

http://jajohnst.si676.si.umich.edu/omeka-s/api/items?key_credential=nj0K6JAqnFgeqyP3Kq40SOhl6fiPBXky&key_identity=yAu5NAEEtJCvGx4s8XgJSgvTFjwvLD6j 200


In [13]:
r.json()

{'@context': 'http://jajohnst.si676.si.umich.edu/omeka-s/api-context',
 '@id': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/items/457',
 '@type': 'o:Item',
 'o:id': 457,
 'o:is_public': True,
 'o:owner': {'@id': 'http://jajohnst.si676.si.umich.edu/omeka-s/api/users/1',
  'o:id': 1},
 'o:resource_class': None,
 'o:resource_template': None,
 'o:thumbnail': None,
 'o:title': None,
 'thumbnail_display_urls': {'large': 'http://jajohnst.si676.si.umich.edu/omeka-s/files/large/a1eff2ad47dd4058d5f5e059003926abdc244dd0.jpg',
  'medium': 'http://jajohnst.si676.si.umich.edu/omeka-s/files/medium/a1eff2ad47dd4058d5f5e059003926abdc244dd0.jpg',
  'square': 'http://jajohnst.si676.si.umich.edu/omeka-s/files/square/a1eff2ad47dd4058d5f5e059003926abdc244dd0.jpg'},
 'o:created': {'@value': '2024-11-19T23:49:55+00:00',
  '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
 'o:modified': {'@value': '2024-11-19T23:49:55+00:00',
  '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
 'o:primary_media': 

So these without Omeka S property IDs will be uploaded, but the properties are not recorded.
Basically, no metadata is recorded.